In [1]:
import tensorflow.compat.v1 as tf
import os
import cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print os.listdir(CIFAR_DIR)

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f)
        return data['data'], data['labels']

# refer to tensoflow.Dataset
class CifarData:
    # shuffle: rearrange data, no dependency bw data
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            
            # collect all data of all labels
            all_data.append(data)
            all_labels.append(labels)
            
                    
        # merge np vectors (items) vertically into a matrix
        self._data = np.vstack(all_data)
        
        # normalize
        self._data = self._data / 127.5 - 1
        
        # merge labels horizonally
        self._labels = np.hstack(all_labels)
        
        print self._data.shape
        print self._labels.shape
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        # current image during the loop
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # random permutate array items
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self. _labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print batch_data
# print batch_labels

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
tf.disable_v2_behavior() 

x = tf.placeholder(tf.float32, [None, 3072])
# [None]
y = tf.placeholder(tf.int64, [None])

# w shape: (3072, 10) 10 labels
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                   initializer=tf.random_normal_initializer(0, 1))
# b shape: (10, )
b = tf.get_variable('b', [10],
                   initializer=tf.constant_initializer(0.0))

# matrix multiply
# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b

# mean square loss
"""
# 1 + e^x --simplify normalize-> api: e^x / sum(e^x)
# [sample estimate[0.01, 0.9, ..., 0.03], []]
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

# y_ -> softmax
# y -> one_hot
# loss = y * log y_
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

# indices
predict = tf.argmax(y_, 1)

# [1, 0, 1, 1, 0, 0]
correct_prediction = tf.equal(predict, y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
init = tf.global_variables_initializer()
batch_size = 10
train_steps = 100000
test_steps = 100

# train_op: train model or only infer
# feed_dict: data
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], feed_dict={
                x: batch_data, 
                y: batch_labels})
        if (i+1) % 500 == 0:
            print '[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val)
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = \
                test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy], 
                    feed_dict = {x: test_batch_data, 
                                 y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print '[Test] Step: %d, acc: %4.5f' % (i+1, test_acc)

[Train] Step: 500, loss: 0.15710, acc: 0.20000
[Train] Step: 1000, loss: 0.17893, acc: 0.10000
[Train] Step: 1500, loss: 0.15711, acc: 0.20000
[Train] Step: 2000, loss: 0.15757, acc: 0.20000
[Train] Step: 2500, loss: 0.13666, acc: 0.30000
[Train] Step: 3000, loss: 0.12180, acc: 0.40000
[Train] Step: 3500, loss: 0.11211, acc: 0.40000
[Train] Step: 4000, loss: 0.10005, acc: 0.50000
[Train] Step: 4500, loss: 0.11394, acc: 0.40000
[Train] Step: 5000, loss: 0.15702, acc: 0.20000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.24000
[Train] Step: 5500, loss: 0.13518, acc: 0.30000
[Train] Step: 6000, loss: 0.15031, acc: 0.20000
[Train] Step: 6500, loss: 0.15977, acc: 0.20000
[Train] Step: 7000, loss: 0.13678, acc: 0.20000
[Train] Step: 7500, loss: 0.13583, acc: 0.30000
[Train] Step: 8000, loss: 0.15583, acc: 0.20000
[Train] Step: 8500, loss: 0.13409, acc: 0.30000
[Train] Step: 9000, loss: 0.13572, acc: 0.30000
[Train] Step: 9500, loss: 0.13078, acc: 0.30000
[Train] Step: 10000, loss: 0.11994

[Train] Step: 76000, loss: 0.07999, acc: 0.60000
[Train] Step: 76500, loss: 0.10000, acc: 0.50000
[Train] Step: 77000, loss: 0.12613, acc: 0.30000
[Train] Step: 77500, loss: 0.09397, acc: 0.50000
[Train] Step: 78000, loss: 0.13869, acc: 0.30000
[Train] Step: 78500, loss: 0.10777, acc: 0.40000
[Train] Step: 79000, loss: 0.12176, acc: 0.40000
[Train] Step: 79500, loss: 0.07516, acc: 0.60000
[Train] Step: 80000, loss: 0.09781, acc: 0.50000
(10000, 3072)
(10000,)
[Test] Step: 80000, acc: 0.37800
[Train] Step: 80500, loss: 0.09900, acc: 0.50000
[Train] Step: 81000, loss: 0.06174, acc: 0.70000
[Train] Step: 81500, loss: 0.11996, acc: 0.40000
[Train] Step: 82000, loss: 0.07872, acc: 0.50000
[Train] Step: 82500, loss: 0.07633, acc: 0.60000
[Train] Step: 83000, loss: 0.13534, acc: 0.30000
[Train] Step: 83500, loss: 0.13395, acc: 0.30000
[Train] Step: 84000, loss: 0.15997, acc: 0.20000
[Train] Step: 84500, loss: 0.07995, acc: 0.60000
[Train] Step: 85000, loss: 0.07183, acc: 0.60000
(10000, 3072)